In [89]:
import numpy as np
import time
import pandas as pd
from scipy import misc
import matplotlib.pyplot as plt
# import tikzplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import pandas as pd
# import cvxpy as cp
from scipy.optimize import fsolve
import torch.optim.lr_scheduler as lr_scheduler
import seaborn as sns
torch.set_default_dtype(torch.float64)


In [90]:
M=2000
ite=10
dim_n=3
T=1
T_C=1
T_F=0.1
delta_c = torch.tensor(T_C/ite)
delta_f=torch.tensor(T_F/ite)
print(delta_c,delta_f)
sigma=0.2
mu=0.3
A0=0.0
gamma=0.0
kappa=1.0
rho=5
X0=10000
neuron_model_psi=20

tensor(0.1000) tensor(0.0100)


In [91]:
model_f= torch.nn.Sequential(
    torch.nn.Linear(dim_n, neuron_model_psi),
    torch.nn.ReLU(),
    #torch.nn.Linear(neuron_model_psi, neuron_model_psi),
    #torch.nn.ReLU(),
    # torch.nn.Linear(neuron_model_psi, neuron_model_psi_2),
    # torch.nn.ReLU(),
    torch.nn.Linear(neuron_model_psi, neuron_model_psi),
    torch.nn.ReLU(),
    torch.nn.Linear(neuron_model_psi,1)
)

In [123]:
#torch.save(model, 'model_N10.pth')
# model_coarse = torch.load('model_N10_best.pth')
model_coarse = torch.nn.Sequential(
    torch.nn.Linear(dim_n, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50,1)
)
model_coarse.load_state_dict(torch.load('model_N10_Arash.pth'))
model_coarse.eval()

Sequential(
  (0): Linear(in_features=3, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=1, bias=True)
)

## Value function

In [124]:
def update(x3,psi,delta):
  t=x3[:,0]+delta
  #print(psi.squeeze(1).shape,x1[:,1].shape)
  D = (x3[:,1]+kappa*psi)*torch.exp(-rho*delta)
  R= x3[:,2]-psi
  #print(R.shape)
  up=torch.cat((t.unsqueeze(1),D.unsqueeze(1),R.unsqueeze(1)),dim=1)
  #print('up shape=', up.shape)
  return up

def loss_func(x2,psi):
  loss=(x2[:,1]*psi+(kappa/2.0)*torch.pow(psi,2))
  return loss

def unit(x1,model,delta):
    psi=model(x1).squeeze(1)
    los=loss_func(x1,psi)
    upd=update(x1,psi,delta)
    #print('unit print=',psi.shape,los.shape,upd.shape)
    return psi,los,upd

def loss_func_total(u,model,delta):
  loss=torch.zeros(M,ite+1)
  psi=torch.zeros(M,ite+1)
  for i in range(ite+1):
    if(i!=ite):
        psi_run,loss_run,u_run=unit(u,model,delta)
        #print('los func=',psi_run.shape,loss_run.shape)
        loss[:,i]=loss_run
        #print(loss)
        psi[:,i]=psi_run
        #print(psi)
        u=u_run
    #print(u)
    else:
    #print(torch.sum(psi,dim=1),R.squeeze(1))
        psi_ter=R.squeeze(1)-torch.sum(psi,dim=1)
        loss_ter=loss_func(u,psi_ter)
      #print('ter',loss_ter.shape)
  #print(torch.sum(loss,dim=1))
  #loss1=torch.sum(loss,dim=1)
  loss1=torch.sum(loss,dim=1)+loss_ter
  #print(loss.shape)
  return torch.mean(loss1)

In [125]:
def testing_coarse(X0,M1):    
    t_test=torch.zeros([M1,1])
    D_test=torch.zeros([M1,1])
    #D_test=torch.FloatTensor(M,1).uniform_(0.1,0.5)   #Price impact D_t
    #R_test=torch.FloatTensor(M1,1).uniform_(50,100)   #remaining balance R_t   #To get a positive solution R_t has to be greater than D_t
    R_test=torch.linspace(X0*0.90, X0*1.1, steps=M1).unsqueeze(1)
    x_test=torch.cat((t_test,D_test,R_test),dim=1)
    #print('Input=',x_test,'\n')
    a=torch.zeros(M1,ite+1)
    #print(a.shape)
    for i in range(ite+1):
      if(i!=ite):
        #print(x_test)
        soln_pred=model_coarse.eval()(x_test).squeeze(1).detach()
        a[:,i]=soln_pred
        x_test=update(x_test,soln_pred,delta_c)
        #print(x_test,'\n')
      else:
        a[:,i]=R_test.squeeze(1)-torch.sum(a,dim=1)

    x1=R_test.squeeze().numpy()
    t_soln=np.zeros((M1,ite+1))
    return x_test,a

In [126]:
def coarse_value(X0,M1):    
    t_test=torch.zeros([M1,1])
    D_test=torch.zeros([M1,1])
    #D_test=torch.FloatTensor(M,1).uniform_(0.1,0.5)   #Price impact D_t
    #R_test=torch.FloatTensor(M1,1).uniform_(50,100)   #remaining balance R_t   #To get a positive solution R_t has to be greater than D_t
    R_test=torch.linspace(X0*0.90, X0*1.1, steps=M1).unsqueeze(1)
    x_test = torch.cat((t_test,D_test,R_test),dim=1)
    x_data = x_test
    #print('Input=',x_test,'\n')
    a=torch.zeros(M1,ite+1)
    #print(a.shape)
    for i in range(ite+1):
      if(i!=ite):
        #print(x_test)
        soln_pred=model_coarse.eval()(x_test).squeeze(1).detach()
        a[:,i]=soln_pred
        x_test=update(x_test,soln_pred,delta_c)
        x_data = torch.cat((x_data,x_test),axis=0)
        #print(x_test,'\n')
      else:
        a[:,i]=R_test.squeeze(1)-torch.sum(a,dim=1)

    # x1=R_test.squeeze().numpy()
    # t_soln=np.zeros((M1,ite+1))
    return x_test,a,x_data

In [127]:
x_c,c_soln,x_data=coarse_value(X0,5)
for x in x_data[-5:][:,0]:
    print("{:.10f}".format(x))

1.0000000000
1.0000000000
1.0000000000
1.0000000000
1.0000000000


In [128]:
a = torch.tensor(1.0/10.0)
print("{:.10f}, {:.10f}, {:.10f}".format(1.0/10.0, torch.tensor(1.0/10.0), a))

0.1000000000, 0.1000000000, 0.1000000000


In [129]:
x_c,c_soln=testing_coarse(X0,5)
# print(c_soln.shape)
# print(x_c,c_soln,torch.sum(a,1))
D_max=torch.max(x_c[:,1])
R_min=torch.min(c_soln)
R_max=torch.max(c_soln)
t=torch.zeros([M,1])
D=torch.FloatTensor(M,1).uniform_(0.0,D_max)
#D=torch.FloatTensor(M,1).uniform_(0.1,0.5)   #Price impact D_t
R=torch.FloatTensor(M,1).uniform_(R_min,R_max)   #remaining balance R_t   #To get a positive solution R_t has to be greater than D_t
x=torch.cat((t,D,R),dim=1)
print('coarse soution=',c_soln[0])

coarse soution= tensor([1623.8970,  639.9296,  639.8604,  640.0034,  640.1218,  640.2160,
         640.2860,  641.0455,  642.7218,  631.9081, 1620.0103])


# Training fine model?

In [130]:
start=time.time()
lr1 = 8e-3
max_epoch = 8000
optimizer = optim.Adam(model_f.parameters(), lr1)
t0 = time.time()
#print(psi.shape)
for epoch in range(max_epoch):
  optimizer.zero_grad()
  cost=loss_func_total(x,model_f,delta_f)
  cost.backward()
  optimizer.step()
  #print(loss.item())
  if (epoch % int(num_epochs/5)== int(num_epochs/5)-1) | (epoch == 0):
    print("At epoch {} the mean cost is {}.  Eproch training time = {:.2E} ms".format(epoch,cost.detach(),1000*(time.time()-t0)))
end=time.time()
print('time elapsed=',end-start)

At epoch 0 the mean error is 1,348,357.7497369533.


In [ ]:
# torch.save(model_f, 'model_N10_fine.pth')
# model_fin = torch.load('model_N10_fine.pth')
model_fin=model_f

In [ ]:
def closed_form(X,ite):
  alpa=np.exp(-rho*T/ite)
  p=np.zeros(ite+1)
  p[0]=X/((ite-1)*(1-alpa)+2)
  p[ite]=X/((ite-1)*(1-alpa)+2)
  for j in range(1,ite):
    p[j]=p[0]*(1-alpa)
  return p

def cost(y,ite):
  cost1=0.0
  D1=0.0
  for i in range(ite+1):
    cost1+=D1*y[i]+(kappa/2.0)*np.power(y[i],2)
    D1=(D1+kappa*y[i])*np.exp(-rho*T/ite)
  return cost1

In [ ]:
def testing_fine_forward(c_soln,ite):    
    f_soln=[]
    c=c_soln[2]
    x_test=torch.tensor([[0.0,0.0,c[0]]])
    for i in range(ite+1):
        if (i!=0):
            x_test[0,2]=x_test[0,2]+c[i] #Adding previous remaining balance to the current balance
        if (i!=ite):         
            for j in range(ite):
                #print(i,x_test)
                soln_pred=model_fin.eval()(x_test).squeeze(1).detach()
                f_soln.append(soln_pred)
                x_test=update(x_test,soln_pred,delta_f)
            #print(x_test[0,2])
    f_soln.append(x_test[0,2].view(1))
    f_soln= np.array([tensor.numpy() for tensor in f_soln]).reshape(len(f_soln))
    return f_soln


# Results from forward pass

In [ ]:
coarse_soln=c_soln[2].numpy()
print('coarse soln',coarse_soln, 'Total=',np.sum(coarse_soln))
fine_soln=testing_fine_forward(c_soln,ite)
true_soln=closed_form(10000,100)
print('\nFine solution=', fine_soln, 'Total=',np.sum(fine_soln))

In [ ]:
t_step1=np.linspace(0,1,num=11)
t_step2=np.linspace(0,1,num=101)
plt.figure(figsize=(14,6))
plt.bar(t_step1,coarse_soln,color='red',width=0.01,label='coarse')
plt.bar(t_step2,fine_soln,color='green',width=0.005,label='fine')
plt.bar(t_step2,true_soln,color='blue',width=0.002,label='true')
plt.xlabel('Time steps')
plt.ylabel('Optimal orders')
plt.title('Forward Pass')
plt.legend()
plt.show()


In [ ]:
# true_cost=cost(true_soln,100)
# fine_cost=cost(fine_soln,100)
# print('True Cost= %0.2f'% true_cost, 'Fine grid cost= %0.2f'% fine_cost)
# error=np.abs(true_cost-fine_cost)
# per_error=100*error/true_cost
# print('Cost Error= %0.2f'% error)
# print('Percentage error= %0.2f'% per_error,'%' )
# D_N=data[0,2].detach().numpy()
# derivative=kappa*(fine_soln[0])-((10000-sum(fine_soln[i] for i in range(len(fine_soln)-1)))+D_N)
# print('\nDerivative of loss with respect to x0 =',derivative)

# Strategy 2 (Adding two consecutive terms to predict the block)

In [ ]:
def testing_fine_first_last(c_soln,ite):    
    f_soln=[]
    c=c_soln[2]
    x_test=torch.tensor([[0.0,0.0,c[0]]])
    #x_test[0,2]=x_test[0,2]+c[ite]
    for i in range(ite):       
        x_test[0,2]=x_test[0,2]+c[i+1] #Adding previous remaining balance to the current balance         
        for j in range(ite):
            #print(i,x_test)
            soln_pred=model_fin.eval()(x_test).squeeze(1).detach()
            f_soln.append(soln_pred)
            x_test=update(x_test,soln_pred,delta_f)
            #print(x_test[0,2])
    f_soln.append(x_test[0,2].view(1))
    f_soln= np.array([tensor.numpy() for tensor in f_soln]).reshape(len(f_soln))
    return f_soln

In [ ]:
# fine_soln=testing_fine_first_last(c_soln,ite)
# true_soln=closed_form(10000,100)
# print('\nFine solution=', fine_soln, fine_soln.shape,'Total=',np.sum(fine_soln))
# print('\nDifference between true and fine=',(fine_soln-true_soln))
# D_N=data[0,2].detach().numpy()
# derivative=kappa*(fine_soln[0])-((10000-sum(fine_soln[i] for i in range(len(fine_soln)-1)))+D_N)
# print('\nDerivative of loss with respect to x0 =',derivative)

In [ ]:
t_step1=np.linspace(0,1,num=11)
t_step2=np.linspace(0,1,num=101)
plt.figure(figsize=(14,6))
#plt.bar(t_step1,coarse_soln,color='red',width=0.01,label='coarse')
plt.bar(t_step2,fine_soln,color='green',width=0.005,label='fine')
plt.bar(t_step2,true_soln,color='yellow',width=0.002,label='true')
plt.xlabel('Time steps')
plt.ylabel('Optimal orders')
plt.title('Forward Pass')
plt.legend()
plt.show()

In [ ]:
# true_cost=cost(true_soln,100)
# fine_cost=cost(fine_soln,100)
# print('True Cost= %0.2f'% true_cost, 'Fine grid cost= %0.2f'% fine_cost)
# error=np.abs(true_cost-fine_cost)
# per_error=100*error/true_cost
# print('Cost Error= %0.2f'% error)
# print('Percentage error= %0.2f'% per_error,'%' )
# D_N=data[0,2].detach().numpy()
# derivative=kappa*(fine_soln[0])-((10000-sum(fine_soln[i] for i in range(len(fine_soln)-1)))+D_N)
# print('\nDerivative of loss with respect to x0 =',derivative)

# Strategy 3: Updated (Adding First and last balance and then doing the forward pass)

In [ ]:
def testing_fine_first_last(c_soln,ite):    
    f_soln=[]
    c=c_soln[2]
    x_test=torch.tensor([[0.0,0.0,c[0]]])
    x_test[0,2]=x_test[0,2]+c[ite]
    for i in range(ite):       
        if (i!=0):
            x_test[0,2]=x_test[0,2]+c[i] #Adding previous remaining balance to the current balance         
        for j in range(ite):
            #print(i,x_test)
            soln_pred=model_fin.eval()(x_test).squeeze(1).detach()
            f_soln.append(soln_pred)
            x_test=update(x_test,soln_pred,delta_f)
            #print(x_test[0,2])
    f_soln.append(x_test[0,2].view(1))
    f_soln= np.array([tensor.numpy() for tensor in f_soln]).reshape(len(f_soln))
    return x_test,f_soln

In [ ]:
data,fine_soln=testing_fine_first_last(c_soln,ite)
D_N=data[0,2].detach().numpy()
derivative_fin=kappa*(fine_soln[0])-((10000-sum(fine_soln[i] for i in range(len(fine_soln)-1)))+D_N)
derivative_true=kappa*(true_soln[0])-((10000-sum(true_soln[i] for i in range(len(true_soln)-1)))+D_N)
print('\nDerivative of loss func with respect to x0 (True soln) =',derivative_true)
print('\nDerivative of loss func with respect to x0 (Fine soln) =',derivative_fin)
true_soln=closed_form(10000,100)
print('\nFine solution=', fine_soln,'Total=',np.sum(fine_soln))
print('\nTrue solution=', true_soln,'Total=',np.sum(true_soln))
print('\nDifference between true and fine=',(fine_soln-true_soln))


In [ ]:
t_step1=np.linspace(0,1,num=11)
t_step2=np.linspace(0,100,num=101)
name='N10_T1_K1_rho5_neo20.eps'
f, ax = plt.subplots(figsize=(7,3),dpi=600)
#plt.bar(t_step1,coarse_soln,color='red',width=0.01,label='coarse')
ax.bar(t_step2,fine_soln,color='green',width=0.8,label='fine-grid solution',alpha=0.5)
ax.bar(t_step2,true_soln,color='red',width=0.4,label='closed-form solution',alpha=0.5)
ax.set_xlabel('Time steps')
ax.set_ylabel('Optimal orders (log-scale)')
f.suptitle('Multiscale Solution for the order of size X={:,}'.format(X0))
ax.legend()
# plt.savefig(name,format='eps')  
ax.set_yscale('log')
# plt.show()
tikzplotlib.save("LOB_Linear.tex")

In [ ]:
true_cost=cost(true_soln,100)
fine_cost=cost(fine_soln,100)
print('True Cost= %0.2f'% true_cost, 'Fine grid cost= %0.2f'% fine_cost)
error=np.abs(true_cost-fine_cost)
per_error=100*error/true_cost
print('Cost Error = %0.2f'% error)
print('Percentage Error = %0.2f'% per_error,'%')

# The derivative of the loss function

In [ ]:
# The derivative of the loss function with respect to x0 = kappa*x0-[kappa(X-(x0+x1+......+x_{N-1})+DN)]
# 